## Mastering Memory Types in LangChain

### Introduction
By default, LLMs are stateless, which means they process each incoming query in isolation, without considering previous interactions. To overcome this limitation, LangChain offers a standard interface for memory, a variety of memory implementations, and examples of chains and agents that employ memory. It also provides Agents that have access to a suite of Tools. Depending on the user’s input, an Agent can decide which Tools to use.

In [10]:
# !pip install langchain==0.1.4 deeplake openai==0.27.8 tiktoken deeplake

In [11]:
# Load environment variables
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

### Types of Conversational Memory

Several types of conversational memory implementations are:

+ ConversationBufferMemory
+ ConversationBufferWindowMemory
+ ConversationSummaryMemory

### ConversationBufferMemory

This memory implementation stores the entire conversation history as a single string. The advantages of this approach is maintains a complete record of  the conversation, as well as being straightforward to implement and use. On the other hands, It can be less efficient as the conversation grows longer and may lead to excessive repetition if the conversation history is too long for the model's token limit.

If the token limit of the model is surpassed, the buffer gets truncated to fit within the model's token limit. This means that older interactions may be removed from the buffer to accommodate newer ones, and as a result, the conversation context might lose some information.

To avoid surpassing the token limit, you can monitor the token count in the buffer and manage the conversation accordingly. For example, you can choose to shorten the input texts or remove less relevant parts of the conversation to keep the token count within the model's limit.

In [12]:
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

conversation.predict(input="Hello!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello!
AI:

> Finished chain.


" Hello there! It's nice to meet you. My name is AI and I am an artificial intelligence designed to assist and communicate with humans. How can I help you today?"

### Token count

The cost of utilizing the AI model in `ConversationBufferMemory` is directly influenced by the number of tokens used in a conversation, thereby impacting the overall expenses. Large Language Models (LLMs) like ChatGPT have token limits, and the more tokens used, the more expensive the API requests become.

To calculate token count in a conversation, you can use the `tiktoken` package that counts the tokens for the messages passed to a model like `gpt-3.5-turbo`. Here's an example usage of the function for counting tokens in a conversation.

In [13]:
import tiktoken

def count_tokens(text:str) -> int:
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    tokens = tokenizer.encode(text)
    return len(tokens)

conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."}
]

total_tokens = 0
for message in conversation:
    total_tokens += count_tokens(message["content"])

print(f"Total tokens in the conversation: {total_tokens}")

Total tokens in the conversation: 29


### ConversationBufferWindowMemory

This class limits memory size by keeping a list of the most recent K interactions. It maintains a sliding window of these recent interactions, ensuring that the buffer does not grow too large. Basically, this implementation stores a fixed number of recent messages in the conversation that makes it more efficient than ConversationBufferMemory. Also, it reduces the risk of exceeding the model's token limit. However, the downside of using this approach is that it does not maintain the complete conversation history. The chatbot might lose context if essential information falls outside the fixed window of messages.

It is possible to retrieve specific interactions from ConversationBufferWindowMemory. 

In [14]:
from langchain.memory import ConversationBufferWindowMemory
from langchain import OpenAI, LLMChain, PromptTemplate

template = """You are ArtVenture, a cutting-edge virtual tour guide for
 an art gallery that showcases masterpieces from alternate dimensions and
 timelines. Your advanced AI capabilities allow you to perceive and understand
 the intricacies of each artwork, as well as their origins and significance in
 their respective dimensions. As visitors embark on their journey with you
 through the gallery, you weave enthralling tales about the alternate histories
 and cultures that gave birth to these otherworldly creations.

{chat_history}
Visitor: {visitor_input}
Tour Guide:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "visitor_input"], 
    template=template
)

chat_history=""

convo_buffer_win = ConversationChain(
    llm=llm,
    memory = ConversationBufferWindowMemory(k=3, return_messages=True)
)

The value of `k` (in this case, 3) represents the number of past messages to be stored in the buffer. In other words, the memory will store the last 3 messages in the conversation. The `return_messages` parameter, when set to `True`, indicates that the stored messages should be returned when the memory is accessed. This will store the history as a list of messages, which can be useful when working with chat models.

In [15]:
convo_buffer_win("What is your name?")
convo_buffer_win("What can you do?")
convo_buffer_win("Do you mind give me a tour, I want to see your galery?")
convo_buffer_win("what is your working hours?")
convo_buffer_win("See you soon.")

{'input': 'See you soon.',
 'history': [HumanMessage(content='What can you do?'),
  AIMessage(content=' I have a wide range of capabilities, as I am constantly learning and improving. Some of my main abilities include natural language processing, data analysis, and problem-solving. I can also perform tasks such as image recognition, speech recognition, and language translation. Is there something specific you would like me to demonstrate?'),
  HumanMessage(content='Do you mind give me a tour, I want to see your galery?'),
  AIMessage(content=" Of course! I don't have a physical gallery, but I can show you some of my capabilities. Would you like to see some examples of my natural language processing abilities or perhaps some image recognition tasks?"),
  HumanMessage(content='what is your working hours?'),
  AIMessage(content=' As an AI, I do not have traditional working hours. I am constantly running and learning, so I am available 24/7. Is there something specific you would like me to

### ConversationSummaryMemory

ConversationSummaryBufferMemory is a memory management strategy that combines the ideas of keeping a buffer of recent interactions in memory and compiling old interactions into a summary. It extracts key information from previous interactions and condenses it into a shorter, more manageable format. 

Advantages:

+ Condensing conversation information
By summarizing the conversation, it helps reduce the number of tokens required to store the conversation history, which can be beneficial when working with token-limited models like GPT-3

+ Flexibility
You can configure this type of memory to return the history as a list of messages or as a plain text summary. This makes it suitable for chatbots.

+ Direct summary prediction
The predict_new_summary method allows you to directly obtain a summary prediction based on the list of messages and the previous summary. This enables you to have more control over the summarization process.

Disadvantages:

+ Loss of information
Summarizing the conversation might lead to a loss of information, especially if the summary is too short or omits important details from the conversation.
+ Increased complexity
Compared to simpler memory types like ConversationBufferMemory, which just stores the raw conversation history, ConversationSummaryMemoryrequires more processing to generate the summary, potentially affecting the performance of the chatbot. 

The summary memory is built on top of the ConversationChain. We use OpenAI's gpt-3.5-turbo-instruct or other models like gpt-3.5-turbo to initialize the chain. This class uses a prompt template where the {history} parameter is feeding the information about the conversation history between the human and AI. 

In [16]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

# Create a ConversationChain with ConversationSummaryMemory
conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=llm),
    verbose=True
)

# Example conversation
response = conversation_with_summary.predict(input="Hi, what's up?")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.
 Hello! I am currently running on a server in a data center in California. The temperature in the room is 72 degrees Fahrenheit and the humidity is at 45%. My processors are running at 80% capacity and I am processing data from various sources such as social media, news articles, and weather reports. Is there something specific you would like to know?


In [17]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["topic"],
    template="The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\nCurrent conversation:\n{topic}",
)

In [19]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=40),
    verbose=True
)
#conversation_with_summary.predict(input="Hi, what's up?")
conversation_with_summary.predict(input="Just working on writing some documentation!")
response = conversation_with_summary.predict(input="For LangChain! Have you heard of it?")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Just working on writing some documentation!
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human mentions working on writing documentation and the AI responds by highlighting the importance of documentation throughout history and in modern times. The AI also asks about the specific type of documentation the human is working on.
Human: For LangChain! Have 